In [1]:
import pandas as pd

In [2]:
ludnosc = []
for i in range(1, 17):
    ludnosc.append(pd.read_excel('../data/raw/gus/LUD_bilans_ludnosci_31-12-2011_fixed.xls', sheet_name=i))
ludnosc=pd.concat(ludnosc).dropna(axis=0)

In [3]:
powiaty = ludnosc[ludnosc[ludnosc.columns[0]].str.startswith('pow.')]

In [4]:
gmin_codes=powiaty.columns[1]
miejskie_powiaty_id = powiaty[powiaty[gmin_codes].str.strip().str.len()>4][gmin_codes].str.strip().values.tolist()
grodzkie_powiaty = powiaty[powiaty[gmin_codes].str.strip().str.len()==4][gmin_codes].str.strip().values.tolist()

In [5]:
ppl = pd.read_excel("../data/raw/gus/l_macierz_2014_03_18-1.xls", dtype=str)

In [6]:
def aggregate_powiaty(kod_gminy):
    kod_gminy = kod_gminy.strip()
    if kod_gminy in miejskie_powiaty_id:
        return kod_gminy
    
    return kod_gminy[:4]

In [7]:
ppl.columns

Index(['Kod terytorialny gminy zamieszkania', 'Kod terytorialny gminy pracy',
       'Liczba osób', 'Województwo zamieszkania', 'Powiat zamieszkania',
       'Gmina zamieszkania', 'Województwo pracy', 'Poawiat pracy',
       'Gmina pracy'],
      dtype='object')

In [9]:
ppl['From[Powiat]'] = ppl['Kod terytorialny gminy zamieszkania'].apply(aggregate_powiaty)
ppl['To[Powiat]'] = ppl['Kod terytorialny gminy pracy'].apply(aggregate_powiaty)
ppl['# ppl moving'] = ppl['Liczba osób'].astype(int)

In [10]:
mobility = ppl.groupby(['From[Powiat]','To[Powiat]'])['# ppl moving'].sum().reset_index()

In [11]:
ludnosc = powiaty[powiaty.columns[1:3]].copy()
ludnosc.columns = ["ID", "Inhabitants"]
ludnosc['ID']=ludnosc['ID'].astype(str).str.strip()
ludnosc = ludnosc.set_index("ID")['Inhabitants']

In [12]:
ludnosc_dict = ludnosc.to_dict()

In [13]:
between_mobility = mobility.copy() #[mobility['From[Powiat]']!= mobility['To[Powiat]']].copy()

In [14]:
ppl.groupby(['From[Powiat]'])['# ppl moving'].sum()

From[Powiat]
0201        6948
0202       10064
0203        9270
0204        3136
0205        4700
           ...  
3217        2926
3218        2069
3261011     3587
3262011     7739
3263011      574
Name: # ppl moving, Length: 379, dtype: int64

In [15]:
src_dict = ppl.groupby(['From[Powiat]'])['# ppl moving'].sum().to_dict()
dest_dict = ppl.groupby(['To[Powiat]'])['# ppl moving'].sum().to_dict()

In [48]:
between_mobility['Inhabitants_From[Powiat]'] = between_mobility['From[Powiat]'].apply(lambda x: ludnosc_dict.get(x, None))
between_mobility['Inhabitants_To[Powiat]'] = between_mobility['To[Powiat]'].apply(lambda x: ludnosc_dict.get(x, None))
between_mobility['Inhabitants_To_From[Powiat]'] = between_mobility['To[Powiat]'].apply(lambda x: ludnosc_dict.get(x, None))

between_mobility['MobilitySurveyedInhabitants_From[Powiat]']  = between_mobility['From[Powiat]'].apply(lambda x: src_dict.get(x, None))
between_mobility['MobilitySurveyedInhabitants_To[Powiat]']   = between_mobility['To[Powiat]'].apply(lambda x: dest_dict.get(x, None))
between_mobility['MobilitySurveyedInhabitants_To_From[Powiat]']   = between_mobility['To[Powiat]'].apply(lambda x: src_dict.get(x, None))

In [49]:
between_mobility['%ppl moving']= between_mobility['# ppl moving'].div(between_mobility['MobilitySurveyedInhabitants_From[Powiat]'],axis=0)

In [50]:
indexed_ = between_mobility.set_index(['From[Powiat]', 'To[Powiat]'])

In [51]:
mobilities_diff = between_mobility[between_mobility['From[Powiat]'] != between_mobility['To[Powiat]']]

In [52]:
mobilities_diff

,From[Powiat],To[Powiat],# ppl moving,Inhabitants_From[Powiat],Inhabitants_To[Powiat],MobilitySurveyedInhabitants_From[Powiat],MobilitySurveyedInhabitants_To[Powiat],%ppl moving,MobilitySurveyedInhabitants_To_From[Powiat]
1,0201,0210,101,90528,56732,6948,2854,0.014537,4222
2,0201,0211,145,90528,106966,6948,8251,0.020869,11605
3,0201,0212,85,90528,47877,6948,2100,0.012234,3590
4,0201,0216,306,90528,63121,6948,15715,0.044041,8008
5,0201,0223,173,90528,121651,6948,17032,0.024899,9270
...,...,...,...,...,...,...,...,...,...
7064,3263011,3064011,27,41516,553564,574,85744,0.047038,24278
7065,3263011,3207,80,41516,48391,574,1357,0.139373,2486
7066,3263011,3214,17,41516,120899,574,3068,0.029617,7378
7067,3263011,3261011,11,41516,109233,574,6649,0.019164,3587


In [53]:
indexed_.loc['0264011','0261011']

# ppl moving                                       80.000000
Inhabitants_From[Powiat]                       631235.000000
Inhabitants_To[Powiat]                          83463.000000
MobilitySurveyedInhabitants_From[Powiat]        12989.000000
MobilitySurveyedInhabitants_To[Powiat]           4774.000000
%ppl moving                                         0.006159
MobilitySurveyedInhabitants_To_From[Powiat]      3095.000000
Name: (0264011, 0261011), dtype: float64

In [68]:
indexed_.loc['0261011', '0264011']

# ppl moving                                      751.000000
Inhabitants_From[Powiat]                        83463.000000
Inhabitants_To[Powiat]                         631235.000000
MobilitySurveyedInhabitants_From[Powiat]         3095.000000
MobilitySurveyedInhabitants_To[Powiat]          59709.000000
%ppl moving                                         0.242649
MobilitySurveyedInhabitants_To_From[Powiat]     12989.000000
Name: (0261011, 0264011), dtype: float64

In [ ]:
Inhabitants_From

In [18]:
between_mobility['MobilitySurveyedInhabitants_From[Powiat]'].describe()

count     7069.000000
mean      9307.922903
std       7128.514458
min        574.000000
25%       4167.000000
50%       7269.000000
75%      11644.000000
max      53361.000000
Name: MobilitySurveyedInhabitants_From[Powiat], dtype: float64

In [113]:
from scipy.optimize import curve_fit
def func(x, a, b, k):
    x1, x2 = x
    return k*(x1**(1+b))*(x2**a) + k*(x1**a)*(x2**(1+b))

xdata = [
    mobilities_diff.dropna()['MobilitySurveyedInhabitants_From[Powiat]'].values,
    mobilities_diff.dropna()['Inhabitants_To[Powiat]'].values
]
ydata = mobilities_diff.dropna()['# ppl moving'].values

popt, pcov = curve_fit(func, xdata, ydata)

In [114]:
popt 

array([ 8.03773960e-01, -1.96175671e-01,  3.59792667e-06])

In [109]:
ppl_stayed = between_mobility[between_mobility['From[Powiat]']==between_mobility['To[Powiat]']][['From[Powiat]', '# ppl moving']].drop_duplicates().set_index(['From[Powiat]'])['# ppl moving']

In [110]:
ppl_moving = between_mobility[between_mobility['From[Powiat]']!=between_mobility['To[Powiat]']].groupby(['From[Powiat]'])['# ppl moving'].sum()

In [111]:
flows = pd.DataFrame([ppl_stayed, ppl_moving]).T#.dropna()

In [112]:
flows

,# ppl moving,# ppl moving
From[Powiat],,
0201,4337.0,2611.0
0202,5043.0,5021.0
0203,1883.0,7387.0
0204,959.0,2177.0
0205,1643.0,3057.0
...,...,...
3063011,NaN,3385.0
3064011,NaN,24278.0
3261011,NaN,3587.0


In [95]:
marginals = flows.div(flows.sum(axis=1), axis=0)
marginals.columns = ['% ppl staying', '% ppl moving']

In [99]:
marginals['# inhabitants'] = pd.Series(ludnosc_dict)

In [101]:
marginals_all = marginals.dropna()

In [104]:
marginals_all.to_excel('./result.xlsx')

In [103]:
ludnosc.shape

(380,)

In [42]:
from scipy.optimize import curve_fit
def func(x, a, b, k):
    x1, x2 = x
    return k*(x1**(1+b))*(x2**a) + k*(x1**a)*(x2**(1+b))

xdata = [
    mobilities_diff.dropna()['Inhabitants_To[Powiat]'].values,
    mobilities_diff.dropna()['Inhabitants_From[Powiat]'].values
]
ydata = mobilities_diff.dropna()['# ppl moving'].values

popt, pcov = curve_fit(func, xdata, ydata)

In [43]:
popt 

array([ 0.48664416, -0.51334907,  0.00109632])

In [8]:
def aggregate_powiaty(kod_gminy):
    kod_gminy = kod_gminy.strip()
    if kod_gminy in miejskie_powiaty_id:
        return kod_gminy
    
    return kod_gminy[:4]

In [10]:
mobilnosc_miedzy_powiatami = mobilnosc_miedzy_gminami[mobilnosc_miedzy_gminami['zrodlo']!=mobilnosc_miedzy_gminami['cel']].copy()

In [11]:
mobilnosc_miedzy_powiatami.shape[0], mobilnosc_miedzy_gminami.shape[0]

(23063, 34986)

In [12]:
mobilnosc_miedzy_powiatami['Liczba osób'] = mobilnosc_miedzy_powiatami['Liczba osób'].astype(int)

In [14]:
ludnosc = powiaty[powiaty.columns[1:3]].copy()
ludnosc.columns = ["ID", "Inhabitants"]
ludnosc['ID']=ludnosc['ID'].astype(str).str.strip()
ludnosc = ludnosc.set_index("ID")['Inhabitants']

In [13]:
wiezba_ruchu = mobilnosc_miedzy_powiatami.groupby(['zrodlo', 'cel'])['Liczba osób'].sum().reset_index()

In [19]:
wiezba_ruchu['zrodlo_inhabitants']=wiezba_ruchu.zrodlo.apply(lambda x: ludnosc.to_dict().get(x))
wiezba_ruchu['cel_inhabitants']=wiezba_ruchu.cel.apply(lambda x: ludnosc.to_dict().get(x))

In [21]:
wiezba_ruchu.columns=['From(Powiat)', 'To(Powiat)', '# of people', 'Inhabitants(From)', 'Inhabitants(To)']

In [25]:
wiezba_ruchu.to_excel('../data/processed/wiezba_ruchu.xlsx')

In [27]:
wiezba_ruchu.to_csv('../data/processed/wiezba_ruchu.csv')

In [30]:
out_ = wiezba_ruchu.groupby('From(Powiat)')['# of people'].sum()
in_ = wiezba_ruchu.groupby('To(Powiat)')['# of people'].sum()

In [31]:
final_data = pd.DataFrame([out_, in_]).T
final_data.columns = ['Outgoing', 'Incoming']
final_data = final_data.merge(ludnosc, left_index=True, right_index=True, how='left').copy()

In [33]:
final_data.dropna().to_excel('../data/processed/per_powiat.xlsx')

In [35]:
final_data.dropna().to_csv('../data/processed/per_powiat.csv')

In [59]:
wiezba_ruchu

,From(Powiat),To(Powiat),# of people,Inhabitants(From),Inhabitants(To)
0,0201,0209,62,90528,54744
1,0201,0210,113,90528,56732
2,0201,0211,95,90528,106966
3,0201,0212,123,90528,47877
4,0201,0216,256,90528,63121
...,...,...,...,...,...
6423,3263011,1465011,23,41516,1708491
6424,3263011,2215,10,41516,201630
6425,3263011,3064011,17,41516,553564
6426,3263011,3207,63,41516,48391


In [36]:
# Params

In [55]:
def func(x, a, b, k):
    x1, x2 = x
    return k*(x1**(1+b))*(x2**a) + k*(x1**a)*(x2**(1+b))

In [68]:
symmetrized = pd.DataFrame(
    {
        'Inhabitants(From)': in_+out_,
        'Inhabitants(To)': in_+out_
    })
symmetrized=symmetrized.merge(ludnosc, left_index=True, right_index=True, how='left')

In [56]:
xdata = [
    wiezba_ruchu.dropna()['Inhabitants(From)'].values,
    wiezba_ruchu.dropna()['Inhabitants(To)'].values
]
ydata = wiezba_ruchu.dropna()['# of people'].values

In [57]:
popt, pcov = curve_fit(func, xdata, ydata)

In [58]:
popt

array([ 0.38112657, -0.61882523,  0.0158683 ])

In [76]:
wiezba_ruchu.set_index(['To(Powiat)','From(Powiat)'])['# of people'].values

array([ 62, 113,  95, ...,  17,  63, 121])

In [77]:
wiezba_ruchu.set_index(['From(Powiat)', 'To(Powiat)'])['# of people']

From(Powiat)  To(Powiat)
0201          0209           62
              0210          113
              0211           95
              0212          123
              0216          256
                           ... 
3263011       1465011        23
              2215           10
              3064011        17
              3207           63
              3262011       121
Name: # of people, Length: 6428, dtype: int64

In [78]:
symmetrized = wiezba_ruchu.set_index(['From(Powiat)', 'To(Powiat)'])['# of people'] + wiezba_ruchu.set_index(['To(Powiat)','From(Powiat)'])['# of people']

In [90]:
matrix = wiezba_ruchu[['From(Powiat)', 'To(Powiat)', '# of people']].pivot('From(Powiat)', 'To(Powiat)', '# of people').fillna(0)

In [84]:
wiezba_ruchu.set_index(['To(Powiat)','From(Powiat)'])['# of people'].to_dict()

{('0209', '0201'): 62,
 ('0210', '0201'): 113,
 ('0211', '0201'): 95,
 ('0212', '0201'): 123,
 ('0216', '0201'): 256,
 ('0223', '0201'): 20,
 ('0225', '0201'): 138,
 ('0226', '0201'): 12,
 ('0261011', '0201'): 39,
 ('0262011', '0201'): 155,
 ('0264011', '0201'): 739,
 ('0265011', '0201'): 20,
 ('0804', '0201'): 17,
 ('0811', '0201'): 10,
 ('1061011', '0201'): 52,
 ('1465011', '0201'): 275,
 ('2477011', '0201'): 12,
 ('3022', '0201'): 12,
 ('3064011', '0201'): 13,
 ('0217', '0202'): 16,
 ('0219', '0202'): 716,
 ('0221', '0202'): 22,
 ('0223', '0202'): 1886,
 ('0224', '0202'): 77,
 ('0226', '0202'): 13,
 ('0261011', '0202'): 10,
 ('0262011', '0202'): 27,
 ('0264011', '0202'): 2068,
 ('0265011', '0202'): 215,
 ('1261011', '0202'): 24,
 ('1438', '0202'): 17,
 ('1465011', '0202'): 176,
 ('1661011', '0202'): 14,
 ('0209', '0203'): 25,
 ('0211', '0203'): 1118,
 ('0216', '0203'): 5110,
 ('0223', '0203'): 23,
 ('0226', '0203'): 20,
 ('0262011', '0203'): 180,
 ('0264011', '0203'): 691,
 ('026501

In [80]:
symmetrized.loc[('0201','0209')]

124

In [82]:
wiezba_ruchu.set_index(['From(Powiat)', 'To(Powiat)']).loc[('0201','0209')]

# of people             62
Inhabitants(From)    90528
Inhabitants(To)      54744
Name: (0201, 0209), dtype: int64

In [83]:
wiezba_ruchu.set_index(['From(Powiat)', 'To(Powiat)']).loc[('0209','0201')]

# of people            202
Inhabitants(From)    54744
Inhabitants(To)      90528
Name: (0209, 0201), dtype: int64

In [ ]:
xdata = [
    wiezba_ruchu.dropna()['Inhabitants(From)'].values,
    wiezba_ruchu.dropna()['Inhabitants(To)'].values
]
ydata = symmetrized.dropna()['# of people'].values